In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
years = [year for year in range(1950, 2021)]

### Apparent Temperature

In [26]:
pattern = []
for year in years:
    climate_year = ddf.read_csv(r"/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Climate_Apparent_Temp/apparent_temp_daily_by_year/AT_daily_" + str(year) + ".csv", dtype={'county': 'object'}).compute().drop(columns={'Unnamed: 0'})
    pattern.append(climate_year)

In [27]:
apparent_temp = pd.concat(pattern)
apparent_temp.head()

,year,month,day,longitude,latitude,coor,fips,AT_mean,AT_min,AT_max
0,1950,1,1,-156.656,58.683,"(58.683, -156.656)",2164,-6.594772,-13.578026,-1.813507
1,1950,1,1,-152.488,57.751,"(57.751, -152.488)",2150,-3.923908,-6.731133,-1.076805
2,1950,1,1,-145.459,62.159,"(62.159, -145.459)",2261,-40.946617,-47.830490,-35.634277
3,1950,1,1,-121.300,38.567,"(38.567, -121.3)",6005,0.109799,-2.873749,2.611900
4,1950,1,1,-120.567,37.383,"(37.383, -120.567)",6043,2.655129,0.452988,7.272987


#### Find the bins: L30, X30_40, X40_50, X50_60, X60_70, X70_80, H80

In [28]:
def rangeUnder30(v):
    if v < 30:
        return 1
    else:
        return 0
    

def range30to40(v):
    if v >= 30 and v < 40:
        return 1
    else:
        return 0

def range40to50(v):
    if v >= 40 and v < 50:
        return 1
    else:
        return 0
    
def range50to60(v):
    if v >= 50 and v < 60:
        return 1
    else:
        return 0
    
def range60to70(v):
    if v >= 60 and v < 70:
        return 1
    else:
        return 0
    
def range70to80(v):
    if v >= 70 and v < 80:
        return 1
    else:
        return 0
    
def rangeOver80(v):
    if v >= 80:
        return 1
    else:
        return 0

In [44]:
apparent_mean = apparent_temp.drop(['AT_min', 'AT_max'], axis = 1)
apparent_mean

,year,month,day,longitude,latitude,coor,fips,AT_mean
0,1950,1,1,-156.656,58.683,"(58.683, -156.656)",2164,-6.594772
1,1950,1,1,-152.488,57.751,"(57.751, -152.488)",2150,-3.923908
2,1950,1,1,-145.459,62.159,"(62.159, -145.459)",2261,-40.946617
3,1950,1,1,-121.300,38.567,"(38.567, -121.3)",6005,0.109799
4,1950,1,1,-120.567,37.383,"(37.383, -120.567)",6043,2.655129
...,...,...,...,...,...,...,...,...
587520,2020,12,31,-75.362,38.690,"(38.69, -75.362)",10005,6.723173
587521,2020,12,31,-72.828,41.510,"(41.51, -72.828)",9007,1.020217
587522,2020,12,31,-72.651,41.735,"(41.735, -72.651)",9003,1.076579
587523,2020,12,31,-72.615,44.533,"(44.533, -72.615)",50019,-2.690137


In [45]:
def formatFips(code):
    return str(code).rjust(5, '0')

In [46]:
def celsius_to_fahrenheit(c):
    f = c * 9/5 + 32
    return f

In [47]:
apparent_mean['fips'] = apparent_mean['fips'].apply(formatFips)

In [48]:
apparent_mean

,year,month,day,longitude,latitude,coor,fips,AT_mean
0,1950,1,1,-156.656,58.683,"(58.683, -156.656)",02164,-6.594772
1,1950,1,1,-152.488,57.751,"(57.751, -152.488)",02150,-3.923908
2,1950,1,1,-145.459,62.159,"(62.159, -145.459)",02261,-40.946617
3,1950,1,1,-121.300,38.567,"(38.567, -121.3)",06005,0.109799
4,1950,1,1,-120.567,37.383,"(37.383, -120.567)",06043,2.655129
...,...,...,...,...,...,...,...,...
587520,2020,12,31,-75.362,38.690,"(38.69, -75.362)",10005,6.723173
587521,2020,12,31,-72.828,41.510,"(41.51, -72.828)",09007,1.020217
587522,2020,12,31,-72.651,41.735,"(41.735, -72.651)",09003,1.076579
587523,2020,12,31,-72.615,44.533,"(44.533, -72.615)",50019,-2.690137


In [49]:
apparent_mean['AT_mean'] = apparent_mean['AT_mean'].apply(celsius_to_fahrenheit)
apparent_mean

,year,month,day,longitude,latitude,coor,fips,AT_mean
0,1950,1,1,-156.656,58.683,"(58.683, -156.656)",02164,20.129411
1,1950,1,1,-152.488,57.751,"(57.751, -152.488)",02150,24.936966
2,1950,1,1,-145.459,62.159,"(62.159, -145.459)",02261,-41.703911
3,1950,1,1,-121.300,38.567,"(38.567, -121.3)",06005,32.197639
4,1950,1,1,-120.567,37.383,"(37.383, -120.567)",06043,36.779232
...,...,...,...,...,...,...,...,...
587520,2020,12,31,-75.362,38.690,"(38.69, -75.362)",10005,44.101711
587521,2020,12,31,-72.828,41.510,"(41.51, -72.828)",09007,33.836390
587522,2020,12,31,-72.651,41.735,"(41.735, -72.651)",09003,33.937842
587523,2020,12,31,-72.615,44.533,"(44.533, -72.615)",50019,27.157753


In [50]:
apparent_mean['AT_under30'] = apparent_mean['AT_mean'].apply(rangeUnder30)
apparent_mean['AT_3040'] = apparent_mean['AT_mean'].apply(range30to40)
apparent_mean['AT_4050'] = apparent_mean['AT_mean'].apply(range40to50)
apparent_mean['AT_5060'] = apparent_mean['AT_mean'].apply(range50to60)
apparent_mean['AT_6070'] = apparent_mean['AT_mean'].apply(range60to70)
apparent_mean['AT_7080'] = apparent_mean['AT_mean'].apply(range70to80)
apparent_mean['AT_over80'] = apparent_mean['AT_mean'].apply(rangeOver80)

In [51]:
apparent_mean

,year,month,day,longitude,latitude,coor,fips,AT_mean,AT_under30,AT_3040,AT_4050,AT_5060,AT_6070,AT_7080,AT_over80
0,1950,1,1,-156.656,58.683,"(58.683, -156.656)",02164,20.129411,1,0,0,0,0,0,0
1,1950,1,1,-152.488,57.751,"(57.751, -152.488)",02150,24.936966,1,0,0,0,0,0,0
2,1950,1,1,-145.459,62.159,"(62.159, -145.459)",02261,-41.703911,1,0,0,0,0,0,0
3,1950,1,1,-121.300,38.567,"(38.567, -121.3)",06005,32.197639,0,1,0,0,0,0,0
4,1950,1,1,-120.567,37.383,"(37.383, -120.567)",06043,36.779232,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587520,2020,12,31,-75.362,38.690,"(38.69, -75.362)",10005,44.101711,0,0,1,0,0,0,0
587521,2020,12,31,-72.828,41.510,"(41.51, -72.828)",09007,33.836390,0,1,0,0,0,0,0
587522,2020,12,31,-72.651,41.735,"(41.735, -72.651)",09003,33.937842,0,1,0,0,0,0,0
587523,2020,12,31,-72.615,44.533,"(44.533, -72.615)",50019,27.157753,1,0,0,0,0,0,0


In [54]:
apparent_bins = apparent_mean.drop(['AT_mean', 'day'], axis = 1)
apparent_bins = apparent_bins.groupby(['year', 'month','longitude', 'latitude', 'coor', 'fips']).sum()
apparent_bins = apparent_bins.reset_index()
apparent_bins

,year,month,longitude,latitude,coor,fips,AT_under30,AT_3040,AT_4050,AT_5060,AT_6070,AT_7080,AT_over80
0,1950,1,-156.656,58.683,"(58.683, -156.656)",02060,31,0,0,0,0,0,0
1,1950,1,-156.656,58.683,"(58.683, -156.656)",02164,31,0,0,0,0,0,0
2,1950,1,-152.488,57.751,"(57.751, -152.488)",02150,29,2,0,0,0,0,0
3,1950,1,-152.133,69.367,"(69.367, -152.133)",02185,11,0,0,0,0,0,0
4,1950,1,-150.097,62.320,"(62.32, -150.097)",02170,31,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
821303,2020,12,-70.267,41.667,"(41.667, -70.267)",25001,16,9,5,1,0,0,0
821304,2020,12,-69.797,44.316,"(44.316, -69.797)",23011,26,3,2,0,0,0,0
821305,2020,12,-69.711,43.964,"(43.964, -69.711)",23015,23,6,1,1,0,0,0
821306,2020,12,-68.367,44.450,"(44.45, -68.367)",23009,22,8,0,0,0,0,0


In [55]:
path = '/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Climate_Apparent_Temp/apparent_temp_bins_by_year/AT_bins_'

In [56]:
def saveBinsFile(df, years, path):
    for year in years:
        df_year = df[df['year'] == year]
        df_year.to_csv(path + str(year) + '.csv')

In [57]:
saveBinsFile(apparent_bins, years, path)